In [ ]:
import mmf_setup;mmf_setup.nbinit()
%pylab inline --no-import-all
from nbimports import * 
from mmf_hfb import tf_completion as tf
from mmf_hfb.FuldeFerrelState import FFState
import itertools
from mmfutils.plot import imcontourf
tf.MAX_DIVISION = 200
plt.figure(figsize(10,4))
clear_output()

# Test Thermodynamic in 1D
* This is the self-consistent method, but not success yet.

In [ ]:
def test_Thermodynamic_1d(mu, dmu, delta0=1, k_c=1000, q=0, dq=0,
                 T=0.0,a=0.8, b=1.2, dx=1e-3):
    """test id case"""
    ff = FFState(mu=mu, dmu=dmu, delta=delta0, q=q, dq=dq, dim=1, k_c=k_c, T=T, 
            fix_g=True, bStateSentinel=True)
    n_a, n_b, e, p, mus = ff.get_ns_p_e_mus_1d(mu=mu, dmu=dmu, delta=delta0, q=q, dq=dq, k_c=k_c, update_g=True)
    # fix the g computed from the last line, pass the effective mu to the following routine to have a  

    # Fixed mu_b by changing mu and dmu with same value , as mu_b = mu - dmu
    # Then dP / dx = n_a
    n_a_1, n_b_1, e1, p1, mus1 = ff.get_ns_p_e_mus_1d(mu=mu+dx/2, dmu=dmu+dx/2, mus_eff=mus, q=q, dq=dq, k_c=k_c)
    n_a_2, n_b_2, e2, p2, mus2 = ff.get_ns_p_e_mus_1d(mu=mu-dx/2, dmu=dmu-dx/2, mus_eff=mus, q=q, dq=dq, k_c=k_c)
    n_a_ = (p1 - p2)/2/dx
    print(f"Expected n_a={n_a}\tNumerical n_a={n_a_}")

    # Fixed mu_a by changing mu and dmu with opposite values , as mu_a = mu + dmu
    # Then dP / dx = n_b
    n_a_3, n_b_3, e3, p3, mus3 = ff.get_ns_p_e_mus_1d(mu=mu+dx/2, dmu=dmu-dx/2, mus_eff=mus, q=q, dq=dq, k_c=k_c)
    n_a_4, n_b_4, e4, p4, mus4 = ff.get_ns_p_e_mus_1d(mu=mu-dx/2, dmu=dmu+dx/2, mus_eff=mus, q=q, dq=dq, k_c=k_c)
    n_b_ = (p3 - p4)/2/dx

    print(f"Expected n_b={n_b}\tNumerical n_b={n_b_}")
    assert np.allclose(n_a, n_a_)
    assert np.allclose(n_b, n_b_)


In [ ]:
test_Thermodynamic_1d(mu=3, dmu=0.5, delta0=1, k_c=100, q=0, dq=0, T=0.0,a=0.8, b=1.2, dx=1e-3)

# Old Version
* In the old version code, we do not have the Hartree term in the energy density

In [ ]:
def test_Thermodynamic(mu, dmu, delta0=1, dim=3, k_c=100, q=0, dq=0, T=0.0,a=0.8, b=1.2, dx=1e-3):
    ff = FFState(mu=mu, dmu=dmu, delta=delta0, q=q, dq=dq, dim=dim, k_c=k_c, T=T, 
            fix_g=True, bStateSentinel=True)
    def get_P(mu, dmu):
        delta = ff.solve(mu=mu, dmu=dmu, q=q, dq=dq, a=0.8*delta0, b=1.2*delta0)
        return ff.get_pressure(mu=mu, dmu=dmu, delta=delta, q=q, dq=dq)

    def get_E_n(mu, dmu):
        E = ff.get_energy_density(mu=mu, dmu=dmu, q=q, dq=dq)
        na, nb = ff.get_densities(mu=mu, dmu=dmu, q=q, dq=dq)
        return E, na, nb

    def get_ns(mu, dmu):
        return ff.get_densities(mu=mu, dmu=dmu, q=q, dq=dq)
    
    n_a, n_b = get_ns(mu, dmu)
    n_a_ = (get_P(mu+dx/2, dmu+dx/2) - get_P(mu-dx/2, dmu - dx/2))/2/dx
    n_b_ = (get_P(mu+dx/2, dmu-dx/2) - get_P(mu-dx/2, dmu + dx/2))/2/dx
    print(f"Expected n_a={n_a.n}\tNumerical n_a={n_a_.n}")
    print(f"Expected n_b={n_b.n}\tNumerical n_b={n_b_.n}")
    assert np.allclose(n_a.n, n_a_.n)
    assert np.allclose(n_b.n, n_b_.n)

In [ ]:
test_Thermodynamic(mu=3, dmu=0.5, delta0=1, dim=1, k_c=100, q=0, dq=0, T=0.0,a=0.8, b=1.2, dx=1e-3)